## Predict all material properties using pre-trained convolutional neural network

This notebook loads the included pre-trained convolutional neural network for predicting all non-zero material properties from the following physics:

<u>__Mechanics__</u>:

* <u>Stiffness Matrix</u>: $C_{11}^{H},\ C_{22}^{H},\ C_{33}^{H},\ C_{44}^{H},\ C_{55}^{H},\ C_{66}^{H},\ C_{12}^{H},\ C_{13}^{H},\ C_{23}^{H}$ 

* <u>Young's Modulus</u>: $E_{11}^{H},\ E_{22}^{H},\ E_{33}^{H}$

* <u>Shear Modulus</u>: $G_{23}^{H},\ G_{13}^{H},\ G_{12}^{H}$

* <u>Poisson's Ratio</u>:	$\nu_{12}^{H},\nu_{13}^{H},\nu_{23}^{H},\nu_{21}^{H},\nu_{31}^{H},\nu_{32}^{H}$	

<u>__Fluid Permeability__</u>: $K_{11}^{H},\ K_{22}^{H},\ K_{33}^{H}$

<u>__Thermal Conductivity__</u>: $\kappa_{11}^{H},\ \kappa_{22}^{H},\ \kappa_{33}^{H}$



<u>Input</u>: A 64x64x64 voxel mesh of an architected material unit cell

<u>Workflow</u>: 
1. Load in selected voxel mesh and trained Convolutional Neural Network
2. Model performs inference on input mesh

<u>Output</u>:
Predicted material property vector.

__NOTE 1__: The material property vector's elements are in the order above

__NOTE 2__: Included model predicts values that have been min-max scaled. Code to un-scale data is included here.


# 1.  __Setup - import packages, define properties and paths__

In [1]:
import pandas as pd
import numpy as np
import json
import glob
import os

# PyTorch deep learning library
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

import torch.utils.data as data
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.optim as optim

# Will automatically select GPU acceleration if hardware is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Custom classes and functions
from ML_workflow_utils_v3.Dataset_Preprocessor import Dataset_Preprocessor as DataP
from ML_workflow_utils_v3.Voxel_Mesh_Utils import Plot_Array
from ML_workflow_utils_v3.Misc_Utils import unscale
from ML_workflow_utils_v3.Model_Weights_Util import convert_state_dict



In [2]:
# For setting the directory references for the entire package
from ML_workflow_utils_v3.PackageDirectories import PackageDirectories as PD   

# This code automatically sets the rootpath as the directory the entire package is contained in, which is then called to initialize the PackageDirectories class below
import os
# check current path if desired
# currentpath = os.getcwd()
# print(currentpath)

os.chdir('../../../')
rootpath = os.getcwd()
# print(rootpath)

# Alternately, rootpath can be set manually
# rootpath = 'filepath/containing/entire/ML_package/'

directory = PD(rootpath = rootpath)


### Material Properties and CNN configuration

#### __Note__: `matprops` and `matprops_by_module` are configured for the pre-trained model that comes with this package. If running inference on a new model trained in notebook 1_1, change these variables to reflect the material properties and their groupings used in training.

In [3]:
#### Material Properties #####
matprops = ['volFrac', 
         'CH_11 scaled', 'CH_22 scaled', 'CH_33 scaled', 'CH_44 scaled', 'CH_55 scaled', 'CH_66 scaled',
         'CH_12 scaled', 'CH_13 scaled','CH_23 scaled',
         'EH_11 scaled', 'EH_22 scaled', 'EH_33 scaled',
         'GH_23 scaled', 'GH_13 scaled', 'GH_12 scaled', 
         'vH_12 scaled', 'vH_13 scaled', 'vH_23 scaled', 'vH_21 scaled', 'vH_31 scaled','vH_32 scaled',
         'KH_11 scaled', 'KH_22 scaled', 'KH_33 scaled', 
         'kappaH_11 scaled', 'kappaH_22 scaled', 'kappaH_33 scaled']

matprops_by_module = [['volFrac',], 
                      ['CH_11 scaled', 'CH_22 scaled', 'CH_33 scaled', 'CH_44 scaled', 'CH_55 scaled', 'CH_66 scaled',],
                      ['CH_12 scaled', 'CH_13 scaled','CH_23 scaled',],
                      ['EH_11 scaled', 'EH_22 scaled', 'EH_33 scaled',],
                      ['GH_23 scaled', 'GH_13 scaled', 'GH_12 scaled',],
                      ['vH_12 scaled', 'vH_13 scaled', 'vH_23 scaled', 'vH_21 scaled', 'vH_31 scaled','vH_32 scaled',],
                      ['KH_11 scaled', 'KH_22 scaled', 'KH_33 scaled',],
                      ['kappaH_11 scaled', 'kappaH_22 scaled', 'kappaH_33 scaled']]

num_props = len(matprops) # for determining the output



In [4]:
# Path for saved weights of trained parameter prediction neural network 
cnn_filepath = directory.convnetpath
cnn_cp = 'Mat_prop_CNN3D_all_props_model_weights.pth'

cp_path = os.path.join(cnn_filepath, cnn_cp)

# If loading a model trained in notebook 1_1, use this code:
# cnn_filepath = os.path.join(directory.nb_1_1_path, 'model_CPs')
# cnn_cp = 'PLACEHOLDER_model_weights.pth' -- change to the actual name of the CP

# cp_path = os.path.join(cnn_filepath, cnn_cp)

In [5]:
# Import model architecture

from ML_workflow_utils_v3.CNN_MatProp_ConvNet_AllProperties import MatProp_CNN3D_all_matprops
cnn = MatProp_CNN3D_all_matprops(matprops_by_module).to(device)

# If using a newly trained model, load as follows:
# from ML_workflow_utils_v3.CNN_Property_Predictor_Multimodule import MatProp_CNN3D_varmod
# cnn = MatProp_CNN3D_varmod(matprops_by_module).to(device)

In [ ]:
"""
This command loads the trained model's weights

The included pre-trained model weights were produced using a multi-GPU training setup, therefore the key of each layer contains "module.", 
which is how torch.nn.DataParallel() creates state dictionaries. The call of "convert_state_dict"
If using more than one GPU for topology production, set "convert_weight_keys_to" to 'parallel'
"""

convert_weight_keys_to = 'non-parallel'

model_weights = convert_state_dict(cp_path, convert_to = convert_weight_keys_to)



cnn.load_state_dict(model_weights)

#### To load from the database, utilize Pandas dataframe filtering functions to select by topology family, unit cell type, or any other characteristic

In [7]:
# Load a 64x64x64 mesh array of a unit cell

db_csv_name = 'topology_multiphysics_database_by_partno.csv'
dbpath = os.path.join(directory.source_data_path, db_csv_name)
db = pd.read_csv(dbpath)

In [8]:
# To print only the columns useful for selecting a topology for property prediction -- topology family, unit cell type, volume fraction, and full part number
sel_cols = ['topology_family', 'cell_type', 'volFrac', 'full PN']

In [ ]:
# Prints selected values. Change cell_type to a different topology to display the part numbers and pick a different unit for material property prediction
cell_type = 'gyroid'
print(db[db.cell_type == cell_type][sel_cols])

In [10]:
pn = '05-2502-0009'#00-0000-000'
pn_file = f'{pn}.npz'

data_folder = directory.voxeltopo_path

pn_path = os.path.join(data_folder, pn_file)

array = np.load(pn_path)['arr_0'] # .npz file loads as a dict-like object, this key accesses the binary array that represents the voxel mesh


In [ ]:
# Plot the array in 3 dimensions if desired
Plot_Array(array, export_png=False, binary=True, show=True)

In [12]:
# Generate predictionn
# expand_dims is required because neural network accepts array of size (batch_size, 1, 68, 68 68). batch_size=1 for predicting the parameters of a single array
array = array.astype(np.float32)
target_tensor = torch.tensor(np.expand_dims(array, axis=(0,1)))
target_predictions = cnn(target_tensor.to(dtype=torch.float32).to(device)).cpu().detach().numpy().T
target_predictions = np.squeeze(target_predictions, axis=-1)


In [ ]:
# Print predictions
for i in range(len(matprops)):
    print(f'Predicted \"{matprops[i]}\"  = {target_predictions[i]:.3f}')

In [ ]:
# Un-scale the predictions and print
unscaled_values = []
for i in range(len(matprops)):
    par = matprops[i].split(' ')[0]
    unscaled = unscale(db, par, target_predictions[i])
    unscaled_values.append(unscaled)
    print(f'{par}')
    print(f'Predicted (as scaled): {target_predictions[i]:.4f}')
    print(f'Un-scaled: \t       {unscaled:.4f}')